### This jupyter notebook is to match face of a given photo with all the faces from photos folder 

In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

c:\Users\Tymof\OneDrive\Desktop\education\Practice\Hackaton_BEST_Competition\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('../data/photos') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


### Saving data into data.pt file

In [4]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

### Matching face id of the given photo with available data from data.pt file

In [6]:

def face_match(img_path, data_path, threshold): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
    
    result = []
    for i in range(len(dist_list)):
        if dist_list[i] < threshold:
            r = {}
            r["name"] = name_list[i]
            r["distance"] = dist_list[i]
            result.append(r)
    return result


result = face_match('../data/test.jpg', 'data.pt', 1.2)

print('Result face matching: name/distance:', result)

Result face matching: name/distance: [{'name': 'roma_skok', 'distance': 1.124752163887024}, {'name': 'tymofii_nasobko', 'distance': 0.8902642130851746}]


In [10]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file, bucket_name):
    """Upload data to a bucket"""

    # Initialize the storage client with your service account credentials
    storage_client = storage.Client.from_service_account_json("cloud_credentials.json")

    # Get the bucket by name
    bucket = storage_client.get_bucket(bucket_name)

    # Create a blob object with the specified name, including the folder path
    blob = bucket.blob(blob_name)

    # Upload the file from the specified path to the blob
    blob.upload_from_filename(path_to_file)

# Example usage:
blob_name = "found/test-image.jpg"
path_to_file = "test.jpg"  # This should be the path to your file
bucket_name = "dataface-hackaton"
upload_to_bucket(blob_name, path_to_file, bucket_name)
id = 10
blob_name = f"in-search/{id}/test-image.jpg"
upload_to_bucket(blob_name, path_to_file, bucket_name)

